# Microsoft Azure Data Manager for Agriculture and NDVI summary workflows into a single custom workflow

In this notebook, we will explain how to connect FarmVibes.AI with [Microsoft Azure Data Manager for Agriculture](https://aka.ms/farmvibesDMA), and provide an example of how to leverage the FarmVibes.AI workflows using [ADMAg for Agri](https://learn.microsoft.com/en-us/rest/api/data-manager-for-agri/). We will demonstrate how to compose the ADMAg and NDVI summary workflows into a single custom workflow, and check the results for the user's agriculture field. The Notebook use ADMAg version <b>2023-11-01-preview</b> for demonstration.

### Micromamba environment setup
To install the required packages, see [this README file](../README.md). You can activate the environment with the following command:


```bash
$ micromamba activate farmvibes-ai
```

## Import required modules

In this example, we will use basic modules for data manipulation (e.g., pandas, yaml, and datetime) and FarmVibes.AI vibe_core modules (client, and data).

In [1]:
import yaml
import pandas as pd
from datetime import datetime
from pprint import pprint

from vibe_core.client import get_default_vibe_client
from vibe_core.data import ADMAgSeasonalFieldInput

## Define Azure Data Manager for Agriculture entities

We will start by providing the parameters that specify the Azure Data Manager for Agriculture connection (e.g., seasonal field, and farmer identifiers). Please, check Microsoft Azure Data Manager for Agriculture [documentation](https://learn.microsoft.com/en-us/rest/api/data-manager-for-agri/) to check how to obtain these fields.

In the next cell, we retrieve the `CLIENT_SECRET` variable from the `data-manager-ag-secret` registered on the FarmVibes.AI cluster. To create a new key on the cluster you may want to use the following command on project's root folder:

```
$ farmvibes-ai local add-secret data-manager-ag-secret <CLIENT_SECRET_VALUE>
```
A message like the following, should be prompted.

`secret/data-manager-ag-secret created`

More details about FarmVibes.AI secrets can be found on the [FarmVibes.AI secrets documentation](https://microsoft.github.io/farmvibes-ai/docfiles/markdown/SECRETS.html).

In [2]:
WORKFLOW_NAME = "data_ingestion/admag/admag_seasonal_field"

BASE_URL = "<BASE_URL>"
CLIENT_ID = "<CLIENT_ID>"
CLIENT_SECRET = "<CLIENT_SECRET>"
AUTHORITY = "<AUTHORITY>"
DEFAULT_SCOPE = "<DEFAULT_SCOPE>"

PARTY_ID = "<PARTY_ID>"
SEASONAL_FIELD_ID="<SEASONAL_FIELD_ID>"

## Create Seasonal Field input

Azure Data Manager for Agriculture uses `party_id` and `seasonal_field_id` to identify a crop during a given season. This triple will be used to create a DataVibe subclass `SeasonalFieldInformation` that contains farm-related operations (e.g., fertilization, harvest, tillage, planting, crop name) that is used as input to the workflow (`data_ingestion/admag/admag_seasonal_field`). 

In [3]:
input_data = ADMAgSeasonalFieldInput(
    party_id=PARTY_ID,
    seasonal_field_id=SEASONAL_FIELD_ID,
)

# Retrieve information from ADMAg

The following cells execute the workflow and retrieve the results from FarmVibes.AI cluster.

In [4]:
client = get_default_vibe_client()

In [5]:
client.document_workflow(WORKFLOW_NAME)

Workflow: data_ingestion/admag/admag_seasonal_field

Description:

Generates SeasonalFieldInformation using ADMAg (Microsoft Azure Data Manager for Agriculture).  
    The workflow creates a DataVibe subclass SeasonalFieldInformation that contains farm-related    
    operations (e.g., fertilization, harvest, tillage, planting, crop name).

Sources:

- admag_input (vibe_core.data.farm.ADMAgSeasonalFieldInput): Unique identifiers for ADMAg       
    seasonal field, and party.

...

- client_secret (default: None): Azure Data Manager for Agriculture client secret. Please visit 
    https://aka.ms/farmvibesDMA to check how to get these credentials.

- authority (default: None): Azure Data Manager for Agriculture authority. Please visit         
    https://aka.ms/farmvibesDMA to check how to get these credentials.

- default_scope (default: None): Azure Data Manager for Agriculture default scope. Please visit 
    https://aka.ms/farmvibesDMA to check how to get these credentials.

Tasks:

- admag_seasonal_field: Establishes the connection with ADMAg and fetches seasonal field        
    information.

In [6]:
run = client.run(
    WORKFLOW_NAME,
    "ADMAg Workflow",
    input_data={"admag_input": input_data},
    parameters={
        "base_url": BASE_URL,
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "authority": AUTHORITY,
        "default_scope": DEFAULT_SCOPE
    }
)

run.monitor()

### Check Workflow Output

Using the workflow output, we can see what happened in the Farm during the season.

In [7]:
run.output['seasonal_field'][0]

SeasonalFieldInformation(id='6ee1f27b-c1b8-4e5a-a5c5-d862745229e4', time_range=(datetime.datetime(2000, 2, 15, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2000, 9, 5, 0, 0, tzinfo=datetime.timezone.utc)), bbox=(-117.04672810633345, 47.03859765371245, -117.04516997816333, 47.039879416641384), geometry={'type': 'MultiPolygon', 'coordinates': [[[[-117.0466947519078, 47.038850194363874], [-117.045889480774, 47.03859765371245], [-117.045889480774, 47.03859765371245], [-117.045889480774, 47.03859765371245], [-117.04516997816333, 47.039188503538426], [-117.04555593651739, 47.039807942872116], [-117.04627543912807, 47.039879416641384], [-117.04672810633345, 47.03963640582586], [-117.0466947519078, 47.038850194363874]]]]}, assets=[], crop_name='Alfalfa', crop_type='annual', properties={'pre_1980': 'Irrigation (Pre 1980s)', 'crp_type': 'None', 'crp_start': '', 'crp_end': '', 'year_1980_2000': 'Irrigated: Annual Crops in Rotation', 'year_1980_2000_tillage': 'Intensive Tillage'}, fertil

## Check field characteristics

Users may want to navigate through the `SeasonalFieldInformation` object to evaluate farming data.

In [8]:
seasonal_field = run.output['seasonal_field'][0]

pprint(f"Seasonal field planting date: {seasonal_field.time_range[0].strftime('%Y/%m/%d')}")
pprint(f"Seasonal field harvest date: {seasonal_field.time_range[1].strftime('%Y/%m/%d')}")

pprint(seasonal_field.geometry)

'Seasonal field planting date: 2000/02/15'
'Seasonal field harvest date: 2000/09/05'
{'coordinates': [[[[-117.0466947519078, 47.038850194363874],
                   [-117.045889480774, 47.03859765371245],
                   [-117.045889480774, 47.03859765371245],
                   [-117.045889480774, 47.03859765371245],
                   [-117.04516997816333, 47.039188503538426],
                   [-117.04555593651739, 47.039807942872116],
                   [-117.04627543912807, 47.039879416641384],
                   [-117.04672810633345, 47.03963640582586],
                   [-117.0466947519078, 47.038850194363874]]]],
 'type': 'MultiPolygon'}


# Composing Workflows

In the following cells, we will evaluate Normalized Vegetation Index (NDVI) for a field during the season, whose information is obtained from Azure Data Manager for Agriculture. To do so, we will use the `farm_ai/agriculture/ndvi_summary` workflow:

In [10]:
client.document_workflow("farm_ai/agriculture/ndvi_summary")

Workflow: farm_ai/agriculture/ndvi_summary

Description:

Calculates NDVI statistics (mean, standard deviation, maximum and minimum) for the input        
    geometry and time range. The workflow retrieves the relevant Sentinel-2 products with Planetary 
    Computer (PC) API, forwards them to a cloud detection model and combines the predicted cloud    
    mask to the mask obtained from the product. The workflow computes the NDVI for each available   
    tile and date, summarizing each with the mean, standard deviation, maximum and minimum values   
    for the regions not obscured by clouds. Finally, it outputs a timeseries with such statistics   
    for all available dates, ignoring heavily-clouded tiles.

Sources:

- user_input (vibe_core.data.core_types.DataVibe): Time range and geometry of interest.

...

- pc_key (default: ): Optional Planetary Computer API key.

Tasks:

- s2: Downloads and preprocesses Sentinel-2 imagery that covers the input geometry and time     
    range, and computes improved cloud masks using cloud and shadow segmentation models.

- compute_ndvi: Computes an index from the bands of an input raster.

- summary_timeseries: Computes the mean, standard deviation, maximum, and minimum values of all 
    regions of the raster considered by the mask and aggregates them into a timeseries.

## Create and execute the custom workflow

Observe the following workflow composed of two tasks (**admag_seasonal_field** and **ndvi_summary**) in the next cell. The first task creates the `SeasonalFieldInformation` object which contains the farming-related data from Azure Data Manager for Agriculture, whereas `ndvi_summary` calculates the NVDI index for this crop during the season (from planting to harvest).

Please, observe the edge mapping **admag_seasonal_field.seasonal_field** to **ndvi_summary.user_input**. This enables the `SeasonalFieldInformation` objects to be used for evaluating NDVI the the field.

In [10]:
custom_workflow = """
name: custom_admag
sources:
  admag_input:
    - admag_seasonal_field.admag_input
sinks:
  seasonal_field: admag_seasonal_field.seasonal_field
  ndvi_summary: ndvi_summary.timeseries
parameters:
  base_url:
  client_id:
  client_secret:
  authority:
  default_scope:
tasks:
  admag_seasonal_field:
    workflow: data_ingestion/admag/admag_seasonal_field
    parameters:
      base_url: "@from(base_url)"
      client_id: "@from(client_id)"
      client_secret: "@from(client_secret)"
      authority: "@from(authority)"
      default_scope: "@from(default_scope)"
  ndvi_summary:
    workflow: farm_ai/agriculture/ndvi_summary
edges:
  - origin: admag_seasonal_field.seasonal_field
    destination:
      - ndvi_summary.user_input
"""

workflow_dict = yaml.safe_load(custom_workflow)

In [11]:
run = client.run(
    workflow_dict,
    "Seasonal Field NDVI Evaluation",
    input_data={"admag_input": input_data},
    parameters={
        "base_url": BASE_URL,
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "authority": AUTHORITY,
        "default_scope": DEFAULT_SCOPE,
    }
)

run.monitor()

For the sake of simplicity, we will print only the harvest information of seasonal field. However, the `SeasonalField` returned object may contain data related to fertilization, organic amendments, tillage, and others.

In [12]:
seasonal_field = run.output['seasonal_field'][0]
pprint(seasonal_field.harvests)

Checking the NDVI summary results:

In [13]:
local_path = run.output["ndvi_summary"][0].assets[0].path_or_url
df = pd.read_csv(local_path)
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df.plot(x="date", y="mean", title="NDVI for SeasonalField", ylabel="NDVI Mean", grid=True)

Checking the results as a DataFrame:

In [14]:
df